In [1]:
from eumap.datasets.eo import GLADLandsat
import sys
import os
import traceback
from pathlib import Path
from urllib3.exceptions import MaxRetryError
import numpy as np

#sys.path.append('./eumap/')

from eumap.raster import save_rasters
from eumap.misc import ttprint
from eumap import parallel
import pandas as pd 
from multiprocessing import Pool
from eumap.misc import find_files, nan_percentile
import shutil


In [2]:
tile_ids = ['014E_59N','006W_37N','008E_45N','005E_52N','024E_43N']
year_list = list(range(2000,2022,1))
bands = ['blue','green','red','NIR','SWIR1','SWIR2','thermal','QA']

In [4]:
def worker_download(year,tile):
    idl = list(range(1,24))
    bands = ['blue','green','red','NIR','SWIR1','SWIR2','thermal','QA']
    username = 'lealparente'
    password = '8IaXGlGEvvlFCA+a'
    glad_landsat = GLADLandsat(username, password, verbose=True)
    data, urls, base_raster = glad_landsat.read(tile, f'{year}-1', f'{year}-23',clear_sky=False)
    
    for i in range(8):
        band = bands[i]
        out_path = [f'/mnt/tupi/Xuemeng_testing/{band}/{year}/{iid}/landsat_{tile}.tif' for iid in idl]
        save_rasters(base_raster, out_path, data[i,:,:,:], n_jobs=20)
    
    return None

In [6]:
for year in year_list:
    inputs = [(year,idd) for idd in tile_ids]
    with Pool(5) as p:
        p.starmap(worker_download, inputs)

In [3]:
# rename

doyc = ['001','017','033','049','065','081','097']
doyo = [1,17,33,49,65,81,97]
doyo = [str(i) for i in doyo]
fndic = dict(zip(doyo,doyc))

# os.rename(old_name, new_name)

In [4]:
files = []
for bs in bands:
    for ys in year_list:
        for ds in doyo:
            # ii = fndic.get(ds)
            # ff = f'/mnt/tupi/Xuemeng_testing/original/{bs}/{ys}/{ii}'
            # if os.path.exists(ff):
            #     continue
            # else:
            #     os.makedirs(ff)
                for ts in tile_ids:
                    files.append(f'/mnt/tupi/Xuemeng_testing/original/{bs}/{ys}/{ds}/landsat_{ts}.tif')
                    
news = [i.replace('/'+i.split('/')[-2]+'/', '/'+ fndic.get(i.split('/')[-2])+'/') for i in files]

In [6]:
for i in range(len(files)):
    if os.path.exists(news[i]):
        continue
    else:
        shutil.copy(files[i],news[i])
        os.remove(files[i])

In [8]:
dirs = []
for bs in bands:
    for ys in year_list:
        for ds in doyo:
            dirs.append(f'/mnt/tupi/Xuemeng_testing/original/{bs}/{ys}/{ds}')

In [13]:
for i in range(len(dirs)):
    if os.path.exists(dirs[i]):
        os.rmdir(dirs[i])

In [27]:
rm = find_files('/mnt/tupi/Xuemeng_testing/original','*landsat*.tif')
for i in rm:
    os.remove(str(i))

In [8]:
rmf = ['/mnt/tupi/Xuemeng_testing/'+i for i in bands]
for i in rmf:
    shutil.rmtree(i)

In [14]:
files = find_files('/mnt/tupi/Xuemeng_testing/original','landsat_*.tif')

In [16]:
len(files)-22*8*5*23

0